In [2]:
import requests
from bs4 import BeautifulSoup
import urllib
from pathlib import Path
import time
from datetime import date

In [60]:
out_folder = Path("html/high_diviends")
out_folder.mkdir(exist_ok=True)

date = date.today().strftime("%Y%m%d")

for page in list(range(1,6)):
    load_url = f"https://info.finance.yahoo.co.jp/ranking/?kd=8&tm=d&vl=a&mk=1&p={page}"
    html = requests.get(load_url)
    filename = f"{date}_{page}.html"
    out_path = out_folder.joinpath(filename)
    with open(out_path, mode="wb") as f:
        f.write(html.content)
    time.sleep

In [63]:
def get_stock_list(date, page):
    filename = f"{date}_{page}.html"
    out_path = out_folder.joinpath(filename)
    with open(out_path, mode="r") as f:
        html = f.read()
        soup = BeautifulSoup(html, "html.parser")
        rankingTable = soup.find(class_="rankingTable")
        stockList = []
        for tr in rankingTable.find_all("tr", class_="rankingTabledata"):
            stock = []
            for data in tr.find_all("td"):
                stock.append(data.text)
            stockList.append(stock)
    return stockList

In [64]:
import pandas as pd
year = "2021"
day = "0123"
date = f"{year}{day}""
columns=['rank','ticker','market','name','date','stock_price','fiscal_month','DPS','dividend','掲示板']
df_concat = pd.DataFrame()

for page in list(range(1,6)):
    stock_list = get_stock_list(date, page)
    df = pd.DataFrame(stock_list, columns=columns)
    df = df.drop("掲示板", axis=1)
    df_concat = pd.concat([df_concat,df], ignore_index=True)
df_concat
# P1 : https://info.finance.yahoo.co.jp/ranking/?kd=8
# P3 : https://info.finance.yahoo.co.jp/ranking/?kd=8&p=5
# detail : https://stocks.finance.yahoo.co.jp/stocks/detail/?code=7860.t

,rank,ticker,market,name,date,value,fiscal_month,DPS,dividend
0,1,7860,東証1部,エイベックス(株),01/22,"1,285",2021/03,121.00,9.42%
1,2,9904,東証2部,(株)ベリテ,01/22,349,2021/03,31.12,8.92%
2,3,6104,東証1部,芝浦機械(株),01/22,"2,381",2021/03,199.30,8.37%
3,4,2914,東証1部,ＪＴ,01/22,"2,048",2020/12,154.00,7.52%
4,5,7494,東証1部,(株)コナカ,01/22,308,2021/09,20.00,6.49%
...,...,...,...,...,...,...,...,...,...
245,246,8917,東証1部,ファースト住建(株),01/22,"1,062",2021/10,43.00,4.05%
246,247,8954,東証,オリックス不動産投資法人,01/22,"169,000",2021/02,"6,840.00",4.05%
247,248,9717,東証1部,(株)ジャステック,01/22,"1,234",2021/11,50.00,4.05%
248,249,9934,東証1部,因幡電機産業(株),01/22,"2,476",2021/03,100.00,4.04%


In [ ]:
# Dynamo DB への保存

In [88]:
# Key フォーマットの調整
df_concat["rank"] = df_concat["rank"].astype(int)

In [89]:
import boto3
from boto3.dynamodb.conditions import Key # Boto 3 SDK によって Attr が自動的に作成される

def load_stocks(stocks:pd.DataFrame, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('HighDividends')
    for index, row in stocks.iterrows():
        
        table.put_item(Item=row.to_dict())

load_stocks(df_concat)